In [170]:
import pandas as pd
import numpy as np
import math
import requests
from bs4 import BeautifulSoup as bs
import time

In [142]:
### create function to find twitter-account with google query
def google_results(name, party):
    
    url = 'https://google.com/search?q={}+{}+twitter'.format(name.replace(" ", "+"), party)
    # ...wait for it....
    time.sleep(1)
    
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    twitter_handle = soup.cite.text
    
    return twitter_handle

In [133]:
nationalrat = pd.read_html("https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Schweizer_Nationalrats_in_der_50._Legislaturperiode")[0]
staenderat = pd.read_html("https://de.wikipedia.org/wiki/Liste_der_Mitglieder_des_Schweizer_St%C3%A4nderats_in_der_50._Legislaturperiode")[0]

In [134]:
nationalrat = nationalrat[["Name", "Partei", "Kanton"]]
nationalrat["Rat"] = "nr"

In [135]:
staenderat = staenderat[["Name", "Partei", "Kanton"]]
staenderat["Rat"] = "sr"

In [138]:
parlament = nationalrat.append(staenderat).reset_index().drop(columns=["index"])

In [143]:
twitter_info_list = []
for idx, row in parlament.iterrows():
    twitter_info_list.append([row["Name"], row["Partei"], google_results(row["Name"], row["Partei"])])

In [144]:
twitter_info = pd.DataFrame(twitter_info_list)
twitter_info.columns = ["name", "party", "url"]

In [146]:
def extract_handle(url):
    if (url.find("twitter") > -1):
        if len(url.split('/')) == 4:
            return(url.split('/')[3].split('?')[0])
        

In [147]:
twitter_info["twitter_handle"] = twitter_info.url.apply(extract_handle)

In [149]:
### export for manual corrections
twitter_info.to_csv("data/twitter_accounts_2019.csv")

In [283]:
### import corrected data
twitter_info_edited = pd.read_csv("data/twitter_accounts_2019_edited.csv", index_col=0)

In [286]:
twitter_users = twitter_info_edited[(twitter_info_edited['twitter_handle'].notnull())]

In [287]:
twitter_users = twitter_users.drop(columns = ["edited"])

In [288]:
import tweepy

In [191]:
consumer_key = "QiFcycH0Dkh2qUMKiEz6KagWy"
consumer_secret = "RJXtTYRe0lI47SXvTVcQZxnihOM7CQW88O9BptgWRWVOGQIlKU"
access_token = "20507198-nXLoFruaOgKrQZiorgJ8SJcISI9DLu1DETgHiiAWq"
access_token_secret = "YO2VQm3frJpVSCcjXFSGynQHeBMbKi3kjj1IkvKcXE3XA"
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [198]:
api = tweepy.API(auth)

In [229]:
user._json["id"]

256863130

In [295]:
### Collect User-Ids
def get_twitter_id(username):
    user = api.get_user(username)
    return user._json["id"]
    
twitter_users["user_id"] = twitter_users.twitter_handle.apply(get_twitter_id)

In [350]:
twitter_users["user_id"] = twitter_users["user_id"].astype(str)

In [ ]:
follower_list = pd.DataFrame(columns=["Name", "Party", "Handle", "Twitter_ID", "Follower_ID"])

In [452]:
for idx, row in twitter_users[89:].iterrows():
    ids = []
    for page in tweepy.Cursor(api.followers_ids, screen_name=row.twitter_handle).pages():
        ids.extend(page)
        time.sleep(60)
    for follower in ids:
        if ((twitter_users['user_id'].astype(str) == str(follower)).any()):
            follower_list = follower_list.append(pd.DataFrame({"Name": row.name,
                                                               "Party": row.party,
                                                               "Handle": row.twitter_handle,
                                                               "Twitter_ID": row.user_id,
                                                               "Follower_ID": str(follower)
                                                              }, index=[0]), ignore_index=True)
    print(row.twitter_handle)

enussbi
paganini_nr
robipanta
gerhardpfister
valerie_piller
hpportmann
quadrantir
lorenzquadri
fregazzi
lukasreimann
maximilian_rei
mathiasreynard
natalierickli
kathyriklin
jfrime
roduitbenjamin
marcoromanoppd
regularytz
wernersalzmann
regine_sauter
schenkersilvia
schneedani67
elisabeth_s_s
uschneiderschue
seilergraf
h_siegenthaler
carlosommaruga
luzistamm
babasteinemann
adelethorens
mtoengi
manueltornare
alinetrede
m_tuena
hansuelivogt
cwasi
fwasserfallen
laurentwehrli
cedricwermuth
adrianwuethrich
zac1967
pirminbischof
pascalebruderer
andreacaroniar
raphaelcomtece
robertcramer_ge
engler_stefan
e_ettlin
JR_Fournier
olifrancais
graberkonrad
ruedinoser
b_haeberli
werner_hoesli
ClaudeJaniak
danieljositsch
christianlevrat
lombardi1956
w_luginbuehl
damian_mueller_
paulrechsteiner
riederbeat
beat_vonlanthen
zanettiroberto
mike_egger
pmbregy


In [595]:
len(twitter_users)

155

In [453]:
follower_list.to_pickle("data/follower_all.pkl")

In [434]:
follower_list = pd.read_pickle("data/follower_all.pkl")

In [455]:
follower_list_red = follower_list[follower_list['Follower_ID'].isin(twitter_users['user_id'].astype(str))]

In [456]:
followers_complete = pd.merge(follower_list_red,
                              twitter_users[["user_id", "name", "twitter_handle", "party"]],
                              left_on="Follower_ID",
                              right_on="user_id",
                              how="left")

In [457]:
# Re-Order and Clean DataFrame
followers_complete = pd.merge(followers_complete,
                             twitter_users[["name", "twitter_handle"]],
                             left_on="Handle",
                             right_on="twitter_handle",
                             how="left")

In [458]:
followers_complete = followers_complete[["name_y", "Name", "Party", "Handle", "Twitter_ID", "Follower_ID", "user_id", "name_x", "twitter_handle_x", "party", "twitter_handle_y"]]


In [459]:
followers_complete = followers_complete.drop(columns=["Name", "twitter_handle_y", "user_id"])

In [460]:
followers_complete.columns = ["user_Name", "user_Party", "user_Handle", "user_ID", "follower_ID", "follower_Name", "follower_Handle", "follower_Party"]

In [461]:
def party_scale(party):
    scale = {
        "sp": 1,
        "gps": 2,
        "evp": 3,
        "glp": 4,
        "cvp": 5,
        "bdp": 6,
        "fdp": 7,
        "lega": 8,
        "mcg": 9,
        "svp": 10
    }
    return scale[party.lower()]

In [462]:
followers_complete["user_Party_Scale"] = followers_complete["user_Party"].apply(party_scale)
followers_complete["follower_Party_Scale"] = followers_complete["follower_Party"].apply(party_scale)

In [463]:
followers_complete["Scale_Diff"] = abs(followers_complete["user_Party_Scale"] - followers_complete["follower_Party_Scale"])

In [554]:
## Fraktionen statt Partien benutzen
followers_complete.user_Party[followers_complete.user_Party == "Lega"] = "SVP"
followers_complete.follower_Party[followers_complete.follower_Party == "Lega"] = "SVP"

followers_complete.user_Party[followers_complete.user_Party == "EVP"] = "CVP"
followers_complete.follower_Party[followers_complete.follower_Party == "EVP"] = "CVP"

followers_complete.user_Party[followers_complete.user_Party == "EVP"] = "CVP"
followers_complete.follower_Party[followers_complete.follower_Party == "EVP"] = "CVP"

/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/sco/miniconda3/lib/python3.7/site-p

In [697]:
followers_complete.to_pickle("data/followers_complete.pkl")

In [637]:
follower_party_data = followers_complete[["follower_Party", "Scale_Diff"]].groupby(["follower_Party"]).sum().reset_index()

In [638]:
follower_party_data["Count"] = followers_complete[["follower_Party", "Scale_Diff"]].groupby(["follower_Party"]).count().reset_index()["Scale_Diff"]

In [639]:
follower_party_data["Distance_of_Parties"] = follower_party_data["Scale_Diff"] / follower_party_data["Count"]

In [640]:
follower_party_data["Amount_of_Foreign_Users"] = followers_complete[["follower_Party", "Scale_Diff"]][followers_complete["user_Party"] != followers_complete["follower_Party"]].groupby(["follower_Party"]).count().reset_index()["Scale_Diff"]

In [641]:
follower_party_data["Share_of_Foreign_Users"] = follower_party_data["Amount_of_Foreign_Users"] / follower_party_data["Count"]

In [642]:
follower_party_data["Number_of_Accounts"] = followers_complete[["follower_Party", "follower_ID"]].groupby(["follower_Party", "follower_ID"]).count().reset_index().groupby(["follower_Party"]).count().reset_index()["follower_ID"]

In [643]:
follower_party_data.sort_values(by=['Share_of_Foreign_Users'], ascending=False)

,follower_Party,Scale_Diff,Count,Distance_of_Parties,Amount_of_Foreign_Users,Share_of_Foreign_Users,Number_of_Accounts
6,glp,1087,398,2.731156,370,0.929648,6
0,BDP,711,287,2.477352,251,0.874564,7
3,GPS,1104,500,2.208000,422,0.844000,10
2,FDP,3368,1170,2.878632,855,0.730769,25
5,SVP,3052,815,3.744785,522,0.640491,31
1,CVP,2376,1287,1.846154,770,0.598291,29
4,SP,5596,2440,2.293443,1228,0.503279,40


In [644]:
user_party_data = followers_complete[["user_Party", "Scale_Diff"]].groupby(["user_Party"]).sum().reset_index()
user_party_data["Count"] = followers_complete[["user_Party", "Scale_Diff"]].groupby(["user_Party"]).count().reset_index()["Scale_Diff"]
user_party_data["Distance_of_Parties"] = user_party_data["Scale_Diff"] / user_party_data["Count"]
user_party_data["Amount_of_Foreign_Users"] = followers_complete[["user_Party", "Scale_Diff"]][followers_complete["user_Party"] != followers_complete["follower_Party"]].groupby(["user_Party"]).count().reset_index()["Scale_Diff"]
user_party_data["Share_of_Foreign_Users"] = user_party_data["Amount_of_Foreign_Users"] / user_party_data["Count"]
user_party_data["Number_of_Accounts"] = followers_complete[["user_Party", "user_ID"]].groupby(["user_Party", "user_ID"]).count().reset_index().groupby(["user_Party"]).count().reset_index()["user_ID"]
user_party_data["Percentage_of_own_party"] = 

In [669]:
user_party_data = followers_complete[["user_Party", "user_ID"]].groupby(["user_Party", "user_ID"]).count().reset_index().groupby(["user_Party"]).count().reset_index()

In [670]:
user_party_data

,user_Party,user_ID
0,BDP,7
1,CVP,30
2,FDP,26
3,GPS,10
4,SP,41
5,SVP,34
6,glp,6


In [693]:
len(followers_complete[(followers_complete.follower_Party == "SP") & (followers_complete.user_Party == "SP")]) / len(followers_complete[(followers_complete.follower_Party == "SP")].follower_ID.unique())

30.3

In [692]:
len(followers_complete[(followers_complete.follower_Party == "SP") & (followers_complete.user_Party != "SP")]) / len(followers_complete[(followers_complete.follower_Party == "SP")].follower_ID.unique())

30.7

In [691]:
len(followers_complete[(followers_complete.follower_Party == "SP")].follower_ID.unique())

40

In [694]:
len(followers_complete[(followers_complete.follower_Party == "SP") & (followers_complete.user_Party == "SP")])

1212

In [696]:
len(followers_complete[(followers_complete.follower_Party == "SP") & (followers_complete.user_Party != "SP")])

1228

In [ ]:
def

In [645]:
user_party_data.sort_values(by=['Share_of_Foreign_Users'], ascending=False)

,user_Party,Scale_Diff,Count,Distance_of_Parties,Amount_of_Foreign_Users,Share_of_Foreign_Users,Number_of_Accounts
6,glp,873,357,2.445378,329,0.921569,6
0,BDP,777,294,2.642857,258,0.877551,7
3,GPS,1350,561,2.406417,483,0.860963,10
2,FDP,3121,1117,2.794091,802,0.717995,26
5,SVP,3248,850,3.821176,557,0.655294,34
1,CVP,2580,1345,1.918216,828,0.615613,30
4,SP,5345,2373,2.252423,1161,0.489254,41


In [646]:
## Tabelle der Parteien exportieren
party_follower = follower_party_data.sort_values(by=['Share_of_Foreign_Users'], ascending=False)[["follower_Party", "Count", "Amount_of_Foreign_Users", "Share_of_Foreign_Users", "Number_of_Accounts"]]
party_user = user_party_data.sort_values(by=['Share_of_Foreign_Users'], ascending=False)[["user_Party", "Count", "Amount_of_Foreign_Users", "Share_of_Foreign_Users", "Number_of_Accounts"]]

with pd.ExcelWriter('data/Rangliste_Fraktionen.xlsx') as writer:
    party_follower.to_excel(writer, sheet_name='Follower', index=False)
    party_user.to_excel(writer, sheet_name='Nutzer', index=False)

In [647]:
user_individual_data = followers_complete[["user_Handle", "user_Name", "Scale_Diff"]].groupby(["user_Handle", "user_Name"]).sum().reset_index()
user_individual_data["Count"] = followers_complete[["user_Handle", "Scale_Diff"]].groupby(["user_Handle"]).count().reset_index()["Scale_Diff"]
user_individual_data["Distance_of_Parties"] = user_individual_data["Scale_Diff"] / user_individual_data["Count"]



In [648]:
user_individual_data = pd.merge(user_individual_data,
                             followers_complete[["user_Handle", "Scale_Diff"]][followers_complete["user_Party"] != followers_complete["follower_Party"]].groupby(["user_Handle"]).count().reset_index(),
                             left_on="user_Handle",
                             right_on="user_Handle",
                             how="left")


In [649]:
user_individual_data.columns = ["user_Handle", "user_Name", "Distance_Total", "Count", "Distance_of_Parties", "Amount_of_Foreign_Users"]
                                

In [650]:
user_individual_data["Share_of_Foreign_Users"] = user_individual_data["Amount_of_Foreign_Users"] / user_individual_data["Count"]

In [651]:
user_individual_data.sort_values(by=['Amount_of_Foreign_Users'], ascending=False)[:10]

,user_Handle,user_Name,Distance_Total,Count,Distance_of_Parties,Amount_of_Foreign_Users,Share_of_Foreign_Users
77,landoltmartin,Martin Landolt,255,95,2.684211,89.0,0.936842
24,bglaettli,Balthasar Glättli,254,93,2.731183,84.0,0.903226
20,beatflach,Beat Flach,223,88,2.534091,83.0,0.943182
31,christamarkwald,Christa Markwalder,318,96,3.312500,79.0,0.822917
37,cwasi,Christian Wasserfallen,288,95,3.031579,76.0,0.800000
72,juerg_grossen,Jürg Grossen,192,78,2.461538,73.0,0.935897
23,bernhardguhl,Bernhard Guhl,204,76,2.684211,70.0,0.921053
10,alinetrede,Aline Trede,209,75,2.786667,67.0,0.893333
17,bastiengirod,Bastien Girod,178,72,2.472222,63.0,0.875000
105,natalierickli,Natalie Rickli,392,83,4.722892,63.0,0.759036


In [652]:
## Top- und Flop-10 vorbereiten
user_individual_data_top_10 = user_individual_data.sort_values(by=['Amount_of_Foreign_Users'], ascending=False)[:10][["user_Name", "Count", "Amount_of_Foreign_Users", "Share_of_Foreign_Users"]]
user_individual_data_flop_10 = user_individual_data.sort_values(by=['Amount_of_Foreign_Users'], ascending=True)[["user_Name", "Count", "Amount_of_Foreign_Users", "Share_of_Foreign_Users"]][user_individual_data.Count > 10][:10]


/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [653]:
follower_individual_data = followers_complete[["follower_Handle", "follower_Name", "Scale_Diff"]].groupby(["follower_Handle", "follower_Name"]).sum().reset_index()
follower_individual_data["Count"] = followers_complete[["follower_Handle", "Scale_Diff"]].groupby(["follower_Handle"]).count().reset_index()["Scale_Diff"]
follower_individual_data["Distance_of_Parties"] = follower_individual_data["Scale_Diff"] / follower_individual_data["Count"]


In [654]:
follower_individual_data = pd.merge(follower_individual_data,
                             followers_complete[["follower_Handle", "Scale_Diff"]][followers_complete["user_Party"] != followers_complete["follower_Party"]].groupby(["follower_Handle"]).count().reset_index(),
                             left_on="follower_Handle",
                             right_on="follower_Handle",
                             how="left")


In [655]:
follower_individual_data.columns = ["follower_Handle", "follower_Name", "Distance_Total", "Count", "Distance_of_Parties", "Amount_of_Foreign_Users"]

In [656]:
follower_individual_data["Share_of_Foreign_Users"] = follower_individual_data["Amount_of_Foreign_Users"] / follower_individual_data["Count"]

In [657]:
follower_individual_data.sort_values(by=['Amount_of_Foreign_Users'], ascending=False)[:10]

,follower_Handle,follower_Name,Distance_Total,Count,Distance_of_Parties,Amount_of_Foreign_Users,Share_of_Foreign_Users
17,beatflach,Beat Flach,441,145,3.041379,140.0,0.965517
33,cwasi,Christian Wasserfallen,457,146,3.130137,122.0,0.835616
20,bernhardguhl,Bernhard Guhl,346,121,2.859504,115.0,0.950413
130,thierryburkart,Thierry Burkart,374,116,3.224138,95.0,0.818966
79,m_aebischer,Matthias Aebischer,495,129,3.837209,92.0,0.713178
29,christianlevrat,Christian Levrat,504,123,4.097561,88.0,0.715447
70,kathrinbertschy,Kathrin Bertschy,263,93,2.827957,88.0,0.946237
59,hhiltpold,Hugues Hiltpold,333,101,3.297030,82.0,0.811881
11,b_haeberli,Brigitte Häberli-Koller,256,102,2.509804,78.0,0.764706
27,christamarkwald,Christa Markwalder,300,93,3.225806,76.0,0.817204


In [658]:
## Top- und Flop-10 vorbereiten
follower_individual_data_top_10 = follower_individual_data.sort_values(by=['Amount_of_Foreign_Users'], ascending=False)[:10][["follower_Name", "Count", "Amount_of_Foreign_Users", "Share_of_Foreign_Users"]]
follower_individual_data_flop_10 = follower_individual_data.sort_values(by=['Amount_of_Foreign_Users'], ascending=True)[["follower_Name", "Count", "Amount_of_Foreign_Users", "Share_of_Foreign_Users"]][follower_individual_data.Count > 10][:10]

/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [659]:
## Tabelle der Nutzer exportieren
with pd.ExcelWriter('data/Rangliste_Parlamentarier.xlsx') as writer:
    follower_individual_data_top_10.to_excel(writer, sheet_name='Follower_Top_10', index=False)
    follower_individual_data_flop_10.to_excel(writer, sheet_name='Follower_Flop_10', index=False)
    user_individual_data_top_10.to_excel(writer, sheet_name='Nutzer_Top_10', index=False)
    user_individual_data_flop_10.to_excel(writer, sheet_name='Nutzer_Flop_10', index=False)

In [260]:
parlament_2 = pd.read_excel("data/mitgliederlist-pc.xls")

WARNING *** file size (139136) not 512 + multiple of sector size (512)


In [262]:
names = parlament_2[["Vorname", "Name"]]

In [264]:
names["Full_Name"] = names["Vorname"] + " " + names["Name"]

/Users/sco/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [281]:
(set(names["Full_Name"]) - set(twitter_info_edited.name))

{'Andrea Gmür-Schönenberger',
 'Andrea Martina Geissbühler',
 'Gregor Rutz',
 'Jean-Pierre Grin',
 'Matthias Samuel Jauslin',
 'Mike Egger',
 'Niklaus-Samuel Gugger',
 'Pascale Bruderer Wyss',
 'Philipp Matthias Bregy',
 'Ruth Humbel',
 'Sandra Sollberger'}

In [282]:
(set(twitter_info_edited.name) - set(names["Full_Name"]))

{'Andrea Geissbühler',
 'Andrea Gmür',
 'Gregor A. Rutz',
 'Jean-Pierre Grin-Hofmann',
 'Matthias Jauslin',
 'Nik Gugger',
 'Pascale Bruderer',
 'Rebecca Ruiz',
 'Ruth Humbel Näf',
 'Sandra Sollberger-Muff'}